<a href="https://colab.research.google.com/github/Trickshotblaster/nn-practices/blob/main/actual_words.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

In [2]:
!pip install tokenizers

In [3]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))

In [4]:
from tokenizers.trainers import BpeTrainer
trainer = BpeTrainer(vocab_size=1000, special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])

In [5]:
from tokenizers.pre_tokenizers import Whitespace
tokenizer.pre_tokenizer = Whitespace()

In [6]:
!wget https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip
!unzip wikitext-103-raw-v1.zip

--2023-10-02 17:10:06--  https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.43.120, 52.217.168.120, 52.217.101.118, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.43.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 191984949 (183M) [application/zip]
Saving to: ‘wikitext-103-raw-v1.zip.2’

wikitext-103-raw-v1 100%[===================>] 183.09M  43.8MB/s    in 4.8s    

2023-10-02 17:10:11 (38.3 MB/s) - ‘wikitext-103-raw-v1.zip.2’ saved [191984949/191984949]

Archive:  wikitext-103-raw-v1.zip
replace wikitext-103-raw/wiki.test.raw? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [7]:
files = [f"wikitext-103-raw/wiki.{split}.raw" for split in ["test", "train", "valid"]]
tokenizer.train(files, trainer)

In [8]:
tokenizer.save("tokenizer-wiki.json")

In [9]:
tokenizer = Tokenizer.from_file("tokenizer-wiki.json")

In [10]:
output = tokenizer.encode("Hello, y'all! How are you 😁 ?")
out_tensor = torch.tensor(output.ids)

In [11]:
out_tensor

tensor([44, 73, 80, 80, 83, 16, 93, 11, 69, 80, 80,  5, 44, 83, 91, 69, 86, 73,
        93, 83, 89,  0, 35])

In [12]:
tokenizer.decode(out_tensor.numpy())

"H e l l o , y ' a l l ! H o w a r e y o u ?"

In [13]:
with open("wikitext-103-raw/wiki.train.raw", 'r') as f:
  text = f.readlines()

In [14]:
text[:10]

[' \n',
 ' = Valkyria Chronicles III = \n',
 ' \n',
 ' Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . \n',
 " The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game 

In [15]:
import random

In [16]:
text_len = len(text) - 1

In [17]:
context_len = 20
emb_dim = 10
tokenizer.enable_truncation(context_len)
tokenizer.enable_padding(direction='left', length=context_len)

In [18]:
vocab_size = tokenizer.get_vocab_size()

pretokenize? aaaa but it takes so long

In [19]:
#text = torch.tensor([tokenizer.encode(t).ids for t in text])

In [20]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [21]:
def make_batch(batch):
  xs = []
  ys = []
  for sentence in batch:
    context = ""
    for word, next in zip(sentence, sentence[1:]):
      context += word
      xs.append(tokenizer.encode(context).ids)
      ys.append(torch.argmax(torch.tensor(tokenizer.encode(word).ids)))
  return torch.tensor(xs).to(device), torch.tensor(ys).to(device)

In [22]:
C = torch.randn((vocab_size, emb_dim)).to(device)
w1 = torch.randn((emb_dim * context_len, 100)).to(device) * 0.1
b1 = torch.randn(100).to(device) * 0.01
w2 = torch.randn((100, vocab_size)).to(device) * 0.1
b2 = torch.randn(vocab_size).to(device) * 0.01

params = [C, w1, b1, w2, b2]
for p in params:
  p.requires_grad = True

In [23]:
import torch.nn.functional as F

In [24]:
num_epochs = 200
batch_size = 32
lr = 0.05

In [ ]:
for epoch in range(num_epochs):
  batch = [text[random.randint(0, text_len)] for i in range(batch_size)]
  #words = torch.tensor([tokenizer.encode_batch(batch)[i].ids for i in range(batch_size)])
  x, y = make_batch(batch)

  emb = C[x].view(-1, emb_dim * context_len)
  l1 = (emb @ w1 + b1).tanh()
  out = (l1 @ w2 + b2).tanh()
  loss = F.cross_entropy(out, y)

  for p in params:
    p.grad = None

  loss.backward()
  for p in params:
    p.data -= p.grad
  if epoch % 10 == 0:
    print(loss.item())

8.878471374511719


In [ ]:
@torch.no_grad()
def forward(text):
  ins = text
  x = torch.tensor(tokenizer.encode(text).ids)
  emb = C[x].view(-1, emb_dim * context_len)
  l1 = (emb @ w1 + b1).tanh()
  out = (l1 @ w2 + b2).tanh()
  out = out.exp()
  out /= out.sum(1, keepdim=True)
  sampled = torch.multinomial(out, num_samples=1)
  return sampled

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
lol = forward('the')
print(lol)
tokenizer.decode_batch(lol.cpu().numpy())

In [ ]:
plt.hist(out.cpu().detach().numpy())